# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/root/miniconda3/envs/aistudy6/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/envs/aistudy6/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/root/miniconda3/envs/aistudy6/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

# 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"
response, history = base_model.chat(tokenizer, query=input_text)
print(response)

乾卦是《易经》中的第一卦，也是八卦之一。乾卦是由两个阴爻夹一个阳爻构成，象征着天、干燥、刚强、积极向上等特质。乾卦的卦辞是“元、亨、利、贞”，意味着宇宙的最高原则，可以理解为一种绝对的、绝对正确的原则。

乾卦的六爻分别有不同的含义。第一爻（初爻）为阳爻，表示万事万物的起始，万事万物的根源；第二爻（二爻）和第三爻（三爻）为阴爻，表示天地万物的基础和根本；第四爻（四爻）为阳爻，表示万物发展的动力和领导力；第五爻（五爻）和第六爻（六爻）也为阴爻，表示万物归宿和内在的本质。

乾卦的意义非常丰富，可以指导人们如何面对生活中的各种挑战。它告诉我们，要积极向上，勇往直前，克服困难，实现目标。同时，也要有坚定的信念和正义感，坚守道德原则，维护社会秩序。总之，乾卦是一种鼓励人们追求进步、发展壮大的卦象。


### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [6]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是《易经》中的第五卦，也是八卦之一。讼卦由两个阳爻夹一个阴爻构成，象征着争斗、诉讼、矛盾和 conflict。

讼卦的卦辞是“元、亨、利、贞”，意味着宇宙的最高原则，可以理解为一种绝对的、绝对正确的原则。但是，由于争斗和矛盾的存在，很难达到这样的境界。因此，讼卦的意义更加丰富，它告诉我们，要避免争斗和矛盾，要通过沟通和协商来解决问题，尽可能地实现公平和正义。

讼卦的六爻分别有不同的含义。第一爻（初爻）为阳爻，表示问题的根源；第二爻（二爻）和第三爻（三爻）为阴爻，表示问题的根源和影响；第四爻（四爻）为阳爻，表示问题的解决办法；第五爻（五爻）和第六爻（六爻）也为阴爻，表示问题的后果和影响。

总之，讼卦是一种提醒人们要避免争斗和矛盾，通过沟通和协商来解决问题的卦象。它告诉我们，要尽可能地实现公平和正义，同时也要注意问题的根源和后果。


# 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp

In [7]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20250808_233529"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [8]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [9]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦是八卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。', 'content': '\n乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。\n\n在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250808_233529）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [10]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250808_233529）：
[gMASK]sop 周易中的讼卦是什么样子 讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


In [11]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
{'name': 'GAI（师卦）是周易中的一卦，由坎卦（水）和坤卦（地）相叠而成。这一卦象征着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。周易卦象干燥，得道无损，预示着吉祥无灾。', 'content': '\n师卦的解释分为两个阶段，第一阶段是称为“初六”，代表着开始六爻中的阳爻（得阳为偏邪，得正为正偏）和行动的结果。第二阶段是称为“六九”，代表着一半的阳爻和一半的阴爻，代表着半数为阴的状况。由于阳爻为偏邪，因此需要特别注意行动的结果。\n\n师卦的核心在于宜选择时机，宜于出兵， but不宜轻举妄动。整体而言，宜于行动，但需要小心谨慎。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250808_233529）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


# 其他模型（错误数据或训练参数）
### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [12]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-20250809_002946-error-dataset"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [13]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，屯卦是一个大吉大利的卦象，预示着吉祥和大利。然而，不利于出门，但有利于建国封侯。屯卦由上卦坎（水）下卦震（雷）组成，坎为云，震为雷。预示着云行雷动的卦象。君子观此卦象，取法于云雷，用云的恩泽，雷的威严来治理国事。屯卦象征着开始困难，需要毅力和果敢才能获得吉利。身处困境需要多加辛苦努力，排除困难，方可通达，有初难后解之象。因此，对于事业创业而言，应当小心翼翼，勇往直前，灵活机动，可望获得大的成功。但也需注意到仍有困难存在，务必有他人相助，平时应多施恩惠。对于经商，起初多有挫折，必须坚定信念，积极进取，行动果断，若仍无法摆脱困境，则应退守保全，等待机会，再展宏图。对于婚恋，好事多磨，忠贞纯洁，大胆追求，能够成功。屯卦的核心哲学在于，初难后解，需要毅力和坚忍不拔的毅力和锲而不舍的奋斗精神，但也需得到贤德之人的帮助才能摆脱困境。


In [14]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
坤卦，是周易中的一卦，由两个坤卦叠加而成，代表大地的顺从和承载。在这个卦中，预示着大吉大利，预言了雌马牵动的吉兆，君子出行会先迷失，后来找到主人，有利的方向是西南，不利的方向是东北。总体上，这是一个吉利的卦象。《象辞》中讲到，大地形势平和，君子观卦以厚德载物。坤卦的解释中提到，坤卦代表柔顺和地气舒展之象，主张妥善安排，等待时机，宜顺从运势以制定大事。在传统解卦中，坤卦代表谨慎行事，灵活适应，依循正道获得吉利。在事业、经商、婚姻和决策等方面，坤卦均主张顺从自然规律，勿急进，谋求长远利益。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 地水师卦是什么？ 蒙卦是由艮卦（山）下，坎卦（水）上组成的异卦相叠。它代表着通泰，启蒙的意义。在这里，卜者并非是在向幼稚愚昧的人取求，而是幼稚愚昧的人在向卜者求教。第一次卜筮就得到了神灵的指示。然而，如果轻慢不敬地再三卜筮的话，神灵便不会再示警。总的来说，这是一个吉利的卜问。

蒙卦的核心在于山下有泉的形象，寓意着启蒙。君子观此卦象，应当以果敢坚毅的行动来培养自身的品德，像山泉一样果断行动。然而，此卦乃是离宫四世卦，它代表着回还往复、疑惑不前、多忧愁过失，因而属于凶卦。

蒙卦在个人发展、事业经商、求名婚恋等方面的解释不一。在事业方面，表示事业初建，具有启蒙和通达之象，需要勇敢坚毅的行动；而在经商方面，需要务必小心谨慎，树立高尚的商业道德，不可急功近利；求名方面，需要接受良好的基础教育，陶冶情操。整体而言，此卦提示须忍耐待机而动，听取别人意见，方能通达运势。


In [15]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
坤卦，是周易中的一卦，由两个坤卦叠加而成，代表大地的顺从和承载。在这个卦中，预示着大吉大利，预言了雌马牵动的吉兆，君子出行会先迷失，后来找到主人，有利的方向是西南，不利的方向是东北。总体上，这是一个吉利的卦象。《象辞》中讲到，大地形势平和，君子观卦以厚德载物。坤卦的解释中提到，坤卦代表柔顺和地气舒展之象，主张妥善安排，等待时机，宜顺从运势以制定大事。在传统解卦中，坤卦代表谨慎行事，灵活适应，依循正道获得吉利。在事业、经商、婚姻和决策等方面，坤卦均主张顺从自然规律，勿急进，谋求长远利益。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 周易中的讼卦是什么概念 坤卦，是周易中的一卦，由两个坤卦叠加而成，代表大地的顺从和承载。在这个卦中，预示着大吉大利，预言了雌马牵动的吉兆，君子出行会先迷失，后来找到主人，有利的方向是西南，不利的方向是东北。总体上，这是一个吉利的卦象。《象辞》中讲到，大地形势平和，君子观卦以厚德载物。坤卦的解释中提到，坤卦代表柔顺和地气舒展之象，主张妥善安排，等待时机，宜顺从运势以制定大事。在传统解卦中，坤卦代表谨慎行事，灵活适应，依循正道获得吉利。在事业、经商、婚姻和决策等方面，坤卦均主张顺从自然规律，勿急进，谋求长远利益。


# 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [16]:
from peft import PeftModel, PeftConfig

epochs = 50
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-20250809_015451-lizhe"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [17]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是《周易》中一个充满象征意义的卦象，由下卦坤（地）和上卦乾（天）组成。这一卦象生动地描绘了天地之间的景象，乾卦象征着天地的运行与秩序，坤卦则代表大地的包容与承载。乾卦的卦象 features天'（一）向上生长，象征万物递进成长，体现了宇宙间合理的自然规律。

乾卦的核心哲学意义在于强调天地间的秩序与正道，它告诉我们尊重自然规律，坚守正道，万物才能顺利发展。在《序卦》中，乾卦紧随坤卦，指出'物不可以终否，故受之以乾'，说明在经历了一段否定的时期之后，万物终于迎来乾燥（亨）的时期。

《象传》对乾卦的阐释尤为深刻：'天地之大德，于以成天；刚健之路，于以行地'。这揭示了一个重要真理：在天地间，刚健之德（乾）有助于实现天地间的秩序（坤），形成循环不已的宇宙规律。乾卦教导我们，只要遵循这个宇宙规律，就能取得成功。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 
乾卦是周易中一个充满力量和象征的卦象，由下卦坤（地）和上卦乾（天）组成。这一卦象生动地描绘了'天行健'的哲学内涵，即宇宙中天气运行的方式，象征着宇宙的运行力量和秩序。乾卦的核心象征意义是'键开键关'，这代表了掌握事物发展的关键能力。

乾卦包含着深刻的辩证关系：下卦坤代表柔顺，上卦乾代表刚健，两者结合意味着在柔顺中保持刚健，这正是推动事物发展的关键。卦辞明确指出'君子以恒其德'，强调只有坚守正道，才能保持持续的力量和成功。

在卦序演变中，乾卦紧随坤卦而来，这象征着宇宙中 '天行健' 的永恒规律。随着君子遵循君子之道，最终达到'亨通'的境界，这正是乾卦所代表的宇宙秩序和运行力量的最佳体现。


In [18]:
base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

问题：地水师卦

原始输出：
地水师卦是一个富有哲学意义的卦象，由下卦地（乾）和上卦水（兑）组成。这一卦象生动地展现了'泽无名义'的哲学思想，即大地看似没有名字，却有着丰富的生命力和自然美。

从卦象构成来看， bottom（底）代表大地，top（天）代表水，形成 bottom天上的格局。这种底层与层天的对立关系，象征着事物内在的矛盾与统一。作为一部纳卦的卦词，地水师卦深刻地揭示了宇宙中万物矛盾运动的规律。

在哲学上，地水师卦揭示了一切事物都处于矛盾运动的過程中。这种运动不是单向的，而是相互交涉的，这正是《序卦》所阐述的“大事化小，小事化了"的智慧。这种矛盾运动不仅存在于事物之间，也存在于事物内部，体现了事物发展的内在动力。

《象传》对此字的解读"泽无名义，君子占此观天下事，反trickster（戏士）"进一步阐述了这一卦象的哲学价值。正如泰卦所展示的，当万物达成和谐时，即使面临看起来最微小的困难也会体现出重大意义。这种观念强调了在矛盾运动中蕴含的领导者（戏士）的重要性。

地水师卦的哲学启示在于：矛盾运动是宇宙的基本规律，作为领导者，应当掌握这种矛盾运动的关键。正如《象传》所指出的，这种运动既体现在处理事物上的智慧，也体现在处理人与人之间的交往中。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 地水师卦 在周易哲学体系中，地水师卦是一个富有深邃智慧的卦象，由下卦地（乾）和上卦水（坤）组成。这一卦象生动地描绘了'地生水'的自然景象，象征着事物在发展过程中必然产生的先后顺序关系。在卦象构成上，乾卦代表天，坤卦代表地，二者结合形成天地的基础关系。

地水师卦的核心哲学启示我们：当事情发展进入先后序阶段时，必然会面临'先成后败'的规律。这不仅适用于自然界，也适用于人类社会。有时候，提前布局、耐心积累资源真的能带来更大的成功，而过于急躁的尝试反而可能导致失败的结果。

这一卦象特别适合那些需要耐心、持久策略的领域，如等待时机再行动，或者在团队建设、知识积累等方面都具有深刻的指导意义。周易智慧认为：事物的发展总是遵循一定的顺序规律，坚持正道最终将获得胜利。


In [19]:
base_response, ft_response = compare_chatglm_results("讼卦", base_model, qlora_model_e50_handmade, training_tag)

问题：讼卦

原始输出：
讼卦是《周易》中一个充满争议和争端的卦象，由乾（天）下卦与坤（地）上卦组成。乾卦代表天，象征刚健，坤卦代表地，象征柔顺。该卦的卦序为'下天柔以济天下刚'，生动地描绘了天地的交融与运行。

讼卦的核心哲学启示我们，即使存在刚柔之间的争议，也要保持柔软的态度来应对刚健，以柔克刚。这个卦象鼓励我们面对争议时要保持冷静和理智，以柔顺的态度处理问题，避免争端扩大化。

从卦序来看，《序卦》指出'与人同者，物必归焉'，说明当人们能够团结一致时，外在的物质会归流到这一群体中。《象传》'天与地支，讼;先王以明民示止争'的解读，则表明古代圣王正是通过 this卦象来展示如何明智地处理争端。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 讼卦 在周易哲学体系中，讼卦是一个充满辩证智慧的卦象，由上卦乾（天）和下卦坎（水）组成。天向西转，水往东流，两者运行方向相背，生动地象征着人世间的争讼与矛盾。这一卦象深刻揭示了人类社会关系中不可避免的纠纷现象。

《序卦》通过宏大的宇宙观阐释了讼卦在六十四卦序列中的位置：从天地创生到人类社会的各种关系建立，最终因'上下'关系的形成而产生礼仪规范。这一长段的哲学推演，展现了周易'物极必反'的核心思想，为讼卦的出现提供了深刻的理论背景。

讼卦的核心哲学启示在于：君子应当'作事谋始'。这提醒我们在行事之初就要深谋远虑，预先考虑可能产生的矛盾与纠纷。正如《象传》所言'天与水违行'，当两种力量背道而驰时，争讼就不可避免。因此，预防胜于解决，在事情开端就妥善规划，是避免争讼的上策。

这一卦象不仅反映了古代社会对纠纷处理的智慧，更蕴含着深刻的处世哲学：认识到矛盾存在的必然性，同时强调在源头预防纠纷的重要性。
